In [53]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import re
from shapely.geometry import Point
from shapely.geometry import shape
from shapely import wkt
from pathlib import Path

In [13]:
BASE_DIR = Path("C:/Projects/Toronto_Waste_Analytics")

shp_path = (
    BASE_DIR
    / "waste_bins"
    / "Solid-waste-in-park-assets-wgs84"
    / "SWMS_PARK_BIN_WGS84.shp"
)
shp_path

WindowsPath('C:/Projects/Toronto_Waste_Analytics/waste_bins/Solid-waste-in-park-assets-wgs84/SWMS_PARK_BIN_WGS84.shp')

In [4]:
gdf_bins = gpd.read_file(shp_path)

In [5]:
gdf_bins.head()

,FID,INSPEC_DAT,PARK_NAME,PARK_LCODE,WARD_NAME,WARD_SCODE,LIT_BTYPE,LIT_BCOUNT,REC_BTYPE,REC_BCOUNT,CON_STATUS,X_COORDI,Y_COORDI,LONGITUDE,LATITUDE,OBJECTID,RID,geometry
0,818,2012-11-22,SCARBOROUGH BLUFFS PARK,0576,Scarborough Southwest (36),36,Litter Domed Lid Toter,1,Recycling Domed Lid Toter,1,Non Seasonal,325581.243,4840277.279,-79.241957,43.704154,10111,1,POINT (-79.24196 43.70415)
1,820,2012-11-22,SCARBOROUGH BLUFFS PARK,0576,Scarborough Southwest (36),36,Litter Domed Lid Toter,1,Recycling Domed Lid Toter,1,Non Seasonal,325656.752,4840501.523,-79.241012,43.706170,10112,2,POINT (-79.24101 43.70617)
2,821,2012-11-22,DUNLOP PARK,1741,Scarborough Southwest (35),35,Litter Domed Lid Toter,1,Recycling Domed Lid Toter,1,Non Seasonal,323580.688,4841266.419,-79.266745,43.713111,10113,3,POINT (-79.26675 43.71311)
3,488,2012-11-06,ADAMS PARK,2265,Scarborough East (44),44,Litter Domed Lid Toter,1,Recycling Domed Lid Toter,1,Non Seasonal,333337.827,4850186.999,-79.145193,43.793097,10001,4,POINT (-79.14519 43.7931)
4,489,2012-11-06,ADAMS PARK,2265,Scarborough East (44),44,Litter Domed Lid Toter,1,Recycling Domed Lid Toter,1,Non Seasonal,333294.677,4850238.051,-79.145726,43.793558,10002,5,POINT (-79.14573 43.79356)


In [6]:
len(gdf_bins)

4768

In [7]:
gdf_bins.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
gdf_bins.geom_type.value_counts()

Point    4768
Name: count, dtype: int64

In [9]:
gdf_bins_utm = gdf_bins.to_crs(epsg=26917)
gdf_bins_utm.crs

<Projected CRS: EPSG:26917>
Name: NAD83 / UTM zone 17N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 84°W and 78°W - onshore and offshore. Canada - Nunavut; Ontario; Quebec. United States (USA) - Florida; Georgia; Kentucky; Maryland; Michigan; New York; North Carolina; Ohio; Pennsylvania; South Carolina; Tennessee; Virginia; West Virginia.
- bounds: (-84.0, 23.81, -78.0, 84.0)
Coordinate Operation:
- name: UTM zone 17N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [10]:
gdf_bins_utm.geometry.x.describe(), gdf_bins_utm.geometry.y.describe()

(count      4768.000000
 mean     628294.125308
 std        8883.670192
 min      610778.153999
 25%      621186.729451
 50%      627267.562684
 75%      634843.440345
 max      651407.225813
 dtype: float64,
 count    4.768000e+03
 mean     4.840881e+06
 std      6.306658e+03
 min      4.826768e+06
 25%      4.835549e+06
 50%      4.840675e+06
 75%      4.846109e+06
 max      4.856581e+06
 dtype: float64)

In [ ]:
pop_groups.columns.tolist()

['YEAR (JULY 1)', 'GENDER', '0 to 14', '15 to 64', '65 Plus', 'TOTAL_CHECK']

In [14]:
BASE_DIR = Path("C:/Projects/Toronto_Waste_Analytics")

stops_path = BASE_DIR / "transit_points" / "TTC Routes and Schedules Data" / "stops.txt"

df_stops = pd.read_csv(stops_path)
df_stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,662,662,Danforth Rd at Kennedy Rd,NaN,43.714379,-79.260939,NaN,NaN,NaN,NaN,NaN,1
1,929,929,Davenport Rd at Bedford Rd,NaN,43.674448,-79.399659,NaN,NaN,NaN,NaN,NaN,1
2,940,940,Davenport Rd at Dupont St,NaN,43.675511,-79.401938,NaN,NaN,NaN,NaN,NaN,2
3,1871,1871,Davisville Ave at Cleveland St,NaN,43.702088,-79.378112,NaN,NaN,NaN,NaN,NaN,1
4,11700,11700,Disco Rd at Attwell Dr,NaN,43.701362,-79.594843,NaN,NaN,NaN,NaN,NaN,1


In [15]:
len(df_stops)

9322

In [16]:
gdf_stops = gpd.GeoDataFrame(
    df_stops,
    geometry=gpd.points_from_xy(df_stops["stop_lon"], df_stops["stop_lat"]),
    crs="EPSG:4326"
)

gdf_stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,662,662,Danforth Rd at Kennedy Rd,NaN,43.714379,-79.260939,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.26094 43.71438)
1,929,929,Davenport Rd at Bedford Rd,NaN,43.674448,-79.399659,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.39966 43.67445)
2,940,940,Davenport Rd at Dupont St,NaN,43.675511,-79.401938,NaN,NaN,NaN,NaN,NaN,2,POINT (-79.40194 43.67551)
3,1871,1871,Davisville Ave at Cleveland St,NaN,43.702088,-79.378112,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.37811 43.70209)
4,11700,11700,Disco Rd at Attwell Dr,NaN,43.701362,-79.594843,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.59484 43.70136)


In [17]:
gdf_stops.geom_type.value_counts(), gdf_stops.crs

(Point    9322
 Name: count, dtype: int64,
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [18]:
gdf_stops_utm = gdf_stops.to_crs(epsg=26917)
gdf_stops_utm.crs

<Projected CRS: EPSG:26917>
Name: NAD83 / UTM zone 17N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 84°W and 78°W - onshore and offshore. Canada - Nunavut; Ontario; Quebec. United States (USA) - Florida; Georgia; Kentucky; Maryland; Michigan; New York; North Carolina; Ohio; Pennsylvania; South Carolina; Tennessee; Virginia; West Virginia.
- bounds: (-84.0, 23.81, -78.0, 84.0)
Coordinate Operation:
- name: UTM zone 17N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
ped_path = BASE_DIR / "pedestrian_proxy" / "Pedestrian Network Data - 4326.csv"

df_ped = pd.read_csv(ped_path)
df_ped.head()

,_id,OBJECTID,ROAD_TYPE,SIDEWALK_CODE,SIDEWALK_DESCRIPTION,CROSSWALK,CROSSWALK_TYPE,PX,PX_TYPE,LENGTH,geometry
0,1,1,Local,7.0,Sidewalk on both sides,NaN,,,,93.867680,"{""coordinates"": [[[-79.5639645086874, 43.73782..."
1,2,2,Collector,7.0,Sidewalk on both sides,NaN,,,,32.546284,"{""coordinates"": [[[-79.567921505888, 43.636179..."
2,3,3,,2.0,Sidewalk on north side only,NaN,,,,117.669206,"{""coordinates"": [[[-79.3779147846482, 43.67845..."
3,4,4,,7.0,Sidewalk on both sides,NaN,,,,223.269710,"{""coordinates"": [[[-79.5185205814575, 43.70501..."
4,5,5,Local,7.0,Sidewalk on both sides,NaN,,,,201.335648,"{""coordinates"": [[[-79.3012214002186, 43.77483..."


In [20]:
df_ped.columns

Index(['_id', 'OBJECTID', 'ROAD_TYPE', 'SIDEWALK_CODE', 'SIDEWALK_DESCRIPTION',
       'CROSSWALK', 'CROSSWALK_TYPE', 'PX', 'PX_TYPE', 'LENGTH', 'geometry'],
      dtype='object')

In [27]:
df_ped["geometry"].head(5)

0    {"coordinates": [[[-79.5639645086874, 43.73782...
1    {"coordinates": [[[-79.567921505888, 43.636179...
2    {"coordinates": [[[-79.3779147846482, 43.67845...
3    {"coordinates": [[[-79.5185205814575, 43.70501...
4    {"coordinates": [[[-79.3012214002186, 43.77483...
Name: geometry, dtype: object

In [29]:
df_ped["geometry"] = df_ped["geometry"].apply(
    lambda g: shape(json.loads(g)) if isinstance(g, str) else shape(g)
)

gdf_ped = gpd.GeoDataFrame(
    df_ped,
    geometry="geometry",
    crs="EPSG:4326"
)

len(gdf_ped), gdf_ped.geom_type.value_counts(), gdf_ped.crs

(87105,
 MultiLineString    87105
 Name: count, dtype: int64,
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [32]:
df_ped["geometry"] = df_ped["geometry"].apply(
    lambda g: shape(json.loads(g)) if isinstance(g, str) else shape(g)
)

gdf_ped = gpd.GeoDataFrame(df_ped.copy(), geometry="geometry", crs="EPSG:4326")

len(gdf_ped), gdf_ped.geom_type.value_counts().head(), gdf_ped.crs

(87105,
 MultiLineString    87105
 Name: count, dtype: int64,
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [33]:
gdf_ped_utm = gdf_ped.to_crs(epsg=26917)
gdf_ped_utm.crs

<Projected CRS: EPSG:26917>
Name: NAD83 / UTM zone 17N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 84°W and 78°W - onshore and offshore. Canada - Nunavut; Ontario; Quebec. United States (USA) - Florida; Georgia; Kentucky; Maryland; Michigan; New York; North Carolina; Ohio; Pennsylvania; South Carolina; Tennessee; Virginia; West Virginia.
- bounds: (-84.0, 23.81, -78.0, 84.0)
Coordinate Operation:
- name: UTM zone 17N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [34]:
gdf_ped_utm.geometry.length.describe()

count    87105.000000
mean        92.154107
std         83.346679
min          4.272812
25%         37.055847
50%         71.759604
75%        114.821695
max       2761.617256
dtype: float64

In [35]:
pop_path = BASE_DIR / "population_context" / "49_census_divisions_mof_population_projections_2024-2051.xlsx"

df_pop = pd.read_excel(pop_path)
df_pop.head()

,"Population Projections for Ontario's 49 Census Divisions by Age and Gender, 2024-2051",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117
0,Sources: Statistics Canada for 2024 and Ontari...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,BROAD AGE GROUPS,NaN,NaN,5-YEAR AGE GROUPS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YEAR (JULY 1),REGION CODE,REGION NAME,GENDER,TOTAL,0 to 14,15 to 64,65 Plus,0 to 4,5 to 9,...,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90+
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_pop = pd.read_excel(pop_path, header=3)

df_pop = df_pop.dropna(how="all")

df_pop.head(), df_pop.columns[:10], df_pop.shape

(      Unnamed: 0   Unnamed: 1   Unnamed: 2 Unnamed: 3 Unnamed: 4  \
 0  YEAR (JULY 1)  REGION CODE  REGION NAME     GENDER      TOTAL   
 2           2024            1      TORONTO       MEN+    1615679   
 3           2024            2      DURHAM        MEN+     393549   
 4           2024            3      HALTON        MEN+     323313   
 5           2024            4        PEEL        MEN+     838114   
 
   BROAD AGE GROUPS Unnamed: 6 Unnamed: 7 5-YEAR AGE GROUPS Unnamed: 9  ...  \
 0          0 to 14   15 to 64    65 Plus            0 to 4     5 to 9  ...   
 2           205631    1179714     230334             64593      70362  ...   
 3            70321     265833      57395             21827      24050  ...   
 4            56338     218954      48021             15094      19043  ...   
 5           127314     598603     112197             39515      42334  ...   
 
   Unnamed: 108 Unnamed: 109 Unnamed: 110 Unnamed: 111 Unnamed: 112  \
 0         81.0         82.0         

In [45]:
df_pop = pd.read_excel(pop_path, header=3)
df_pop = df_pop.dropna(how="all")

In [46]:
df_pop.columns[:10]

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'BROAD AGE GROUPS', 'Unnamed: 6', 'Unnamed: 7', '5-YEAR AGE GROUPS',
       'Unnamed: 9'],
      dtype='object')

In [55]:
raw = pd.read_excel(pop_path, header=None)

def row_has(row, pattern):
    s = row.astype(str).str.upper()
    return s.str.contains(pattern, na=False).any()

mask = raw.apply(lambda r: row_has(r, r"YEAR\s*\(JULY\s*1\)") and row_has(r, r"REGION\s*CODE"), axis=1)

if not mask.any():
     mask = raw.apply(lambda r: row_has(r, r"YEAR\s*\(JULY\s*1\)"), axis=1)

header_row = mask.idxmax()
print("Header detect in the row:", header_row)
print("It is row:")
display(raw.iloc[header_row])



Header detect in the row: 4
It is row:


0      YEAR (JULY 1)
1        REGION CODE
2        REGION NAME
3             GENDER
4              TOTAL
           ...      
113             86.0
114             87.0
115             88.0
116             89.0
117              90+
Name: 4, Length: 118, dtype: object

In [57]:
df_pop = raw.copy()
df_pop.columns = (
    raw.iloc[header_row]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df_pop = df_pop.iloc[header_row + 1:].reset_index(drop=True)
df_pop = df_pop.dropna(how="all")

df_pop.columns = (
    pd.Index(df_pop.columns)
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df_pop.shape, df_pop.columns[:10]

((4116, 118),
 Index(['YEAR (JULY 1)', 'REGION CODE', 'REGION NAME', 'GENDER', 'TOTAL',
        '0 to 14', '15 to 64', '65 Plus', '0 to 4', '5 to 9'],
       dtype='object', name=4))

In [58]:
df_pop[["YEAR (JULY 1)", "REGION CODE", "REGION NAME", "GENDER", "TOTAL"]].head()

4,YEAR (JULY 1),REGION CODE,REGION NAME,GENDER,TOTAL
1,2024,1,TORONTO,MEN+,1615679
2,2024,2,DURHAM,MEN+,393549
3,2024,3,HALTON,MEN+,323313
4,2024,4,PEEL,MEN+,838114
5,2024,5,YORK,MEN+,632807


In [59]:
df_pop["REGION NAME"].unique()[:20], df_pop["REGION NAME"].nunique()

(array(['TORONTO', 'DURHAM ', 'HALTON ', 'PEEL ', 'YORK ', 'BRANT ',
        'DUFFERIN ', 'HALDIMAND-NORFOLK  ', 'HALIBURTON ', 'HAMILTON',
        'MUSKOKA  ', 'NIAGARA ', 'NORTHUMBERLAND ', 'PETERBOROUGH ',
        'SIMCOE ', 'KAWARTHA LAKES', 'WATERLOO ', 'WELLINGTON ', 'OTTAWA',
        'FRONTENAC '], dtype=object),
 49)

In [60]:
df_toronto = df_pop[df_pop["REGION NAME"].str.upper().eq("TORONTO")].copy()
df_toronto[["YEAR (JULY 1)", "GENDER", "TOTAL"]].head(), df_toronto.shape

(4   YEAR (JULY 1) GENDER    TOTAL
 1            2024   MEN+  1615679
 50           2025   MEN+  1594069
 99           2026   MEN+  1570568
 148          2027   MEN+  1551565
 197          2028   MEN+  1538860,
 (84, 118))

In [61]:
tor_total_by_year = (
    df_toronto
    .groupby("YEAR (JULY 1)", as_index=False)["TOTAL"]
    .sum()
    .sort_values("YEAR (JULY 1)")
)
tor_total_by_year.head()


,YEAR (JULY 1),TOTAL
0,2024,6546238
1,2025,6467478
2,2026,6381774
3,2027,6312834
4,2028,6266972


In [62]:
df_toronto_24_25 = (
    df_pop[
        (df_pop["REGION NAME"].str.upper() == "TORONTO") &
        (df_pop["YEAR (JULY 1)"].isin([2024, 2025]))
    ]
    .copy()
)

df_toronto_24_25.shape

(6, 118)

In [64]:
pop_groups["TOTAL_CHECK"] = (
    pop_groups["0 to 14"] +
    pop_groups["15 to 64"] +
    pop_groups["65 Plus"]
)

pop_groups

4,YEAR (JULY 1),GENDER,0 to 14,15 to 64,65 Plus,TOTAL_CHECK
1,2024,MEN+,205631,1179714,230334,1615679
50,2025,MEN+,201868,1155309,236892,1594069
1373,2024,WOMEN+,195649,1167354,294437,1657440
1422,2025,WOMEN+,192297,1146237,301136,1639670
2745,2024,TOTAL BOTH GENDERS,401280,2347068,524771,3273119
2794,2025,TOTAL BOTH GENDERS,394165,2301546,538028,3233739


In [67]:
out_path = BASE_DIR / "population_context" / "population_toronto_2024_2025_age_groups.csv"
pop_groups.to_csv(out_path, index=False)

In [70]:
pop_groups_year = (
    pop_groups
    .groupby("YEAR (JULY 1)", as_index=False)[
        ["0 to 14", "15 to 64", "65 Plus", "TOTAL_CHECK"]
    ]
    .sum()
)

pop_groups_year

4,YEAR (JULY 1),0 to 14,15 to 64,65 Plus,TOTAL_CHECK
0,2024,802560,4694136,1049542,6546238
1,2025,788330,4603092,1076056,6467478


In [71]:
out_path = BASE_DIR / "population_context" / "population_toronto_2024_2025_age_groups.csv"
pop_groups_year.to_csv(out_path, index=False)

out_path

WindowsPath('C:/Projects/Toronto_Waste_Analytics/population_context/population_toronto_2024_2025_age_groups.csv')

In [2]:
## Notes for the Team
# This notebook is for **data overview and validation only**
# No cleaning or transformations are applied here
# Datasets are expected to be placed locally following the repository structure
# Further cleaning and analysis should be done in separate notebooks